In [13]:
import os
import numpy as np
import pandas as pd

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)

BASE_PATH = "/Users/agarwalh/DSIP/Project/DSIP_Project"
EVAL_DIR = os.path.join(BASE_PATH, "Preprocessing_FE_dataset")

In [14]:
eval_path = os.path.join(EVAL_DIR, "eval_blend_int_valid.npz")
data = np.load(eval_path)

y_valid = data["y_valid"]
p_valid = data["p_blend_int"]

y_valid.shape, p_valid.shape, float(y_valid.mean())

((61503,), (61503,), 0.08072776937710356)

In [15]:
thresholds = np.linspace(0.01, 0.99, 99)

records = []
for t in thresholds:
    y_pred = (p_valid >= t).astype(int)
    prec = precision_score(y_valid, y_pred, zero_division=0)
    rec  = recall_score(y_valid, y_pred, zero_division=0)
    f1   = f1_score(y_valid, y_pred, zero_division=0)

    records.append(
        {"threshold": t, "precision": prec, "recall": rec, "f1": f1}
    )

thresh_df = pd.DataFrame(records)
thresh_df.head()

,threshold,precision,recall,f1
0,0.01,0.084036,0.995166,0.154985
1,0.02,0.097700,0.965760,0.177448
2,0.03,0.114656,0.924270,0.204005
3,0.04,0.130602,0.873917,0.227244
4,0.05,0.145943,0.827392,0.248120


In [16]:
best_f1_row = thresh_df.sort_values("f1", ascending=False).iloc[0]
t_f1 = best_f1_row["threshold"]

t_f1, best_f1_row[["f1", "precision", "recall"]].to_dict()

(np.float64(0.15000000000000002),
 {'f1': 0.3429000751314801,
  'precision': 0.27345715997603354,
  'recall': 0.45961732124874116})

In [17]:
y_pred_f1 = (p_valid >= t_f1).astype(int)

print("=== Threshold maximizing F1 ===")
print("Threshold:", t_f1)
print("Confusion matrix:")
print(confusion_matrix(y_valid, y_pred_f1))
print("\nClassification report:")
print(classification_report(y_valid, y_pred_f1, digits=4))

=== Threshold maximizing F1 ===
Threshold: 0.15000000000000002
Confusion matrix:
[[50475  6063]
 [ 2683  2282]]

Classification report:
              precision    recall  f1-score   support

           0     0.9495    0.8928    0.9203     56538
           1     0.2735    0.4596    0.3429      4965

    accuracy                         0.8578     61503
   macro avg     0.6115    0.6762    0.6316     61503
weighted avg     0.8950    0.8578    0.8737     61503

